### Answering Business Questions using SQL and Pandas

In this project, I'm going to practice using my SQL skills to answer business questions. This is a project extracted from the **`DataQuest: Data Engineer Track`**

I'll be using the Chinook database.  The Chinook database contains information about a fictional digital music shop - kind of like a mini-iTunes store. The Chinook database contains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. The Chinook database is provided as a SQLite database file called chinook.db. 

I'll also be using a bit of pandas in this project to write queries that produce dataframe outputs.

I'll be posting the questions and answering them using my code


In [1]:
import pandas as pd
import sqlite3

I'll be using `context managers` to manage my connection to the database because this is a very efficient way to ensure I don't mess up the database by accidental changes and the likes.

I'll be using the `with` context manager.

I want to create functions that will make it easier for me to do some repeated tasks like returning a dataframe from the results of a query, executing queries that don't return functions like `CREATE VIEW` and I also need a function that returns the list of tables and views in my database

In [2]:
# A function that runs my queries and returns a pandas dataframe of the results

def run_query(q):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(q, conn)

In [3]:
# A function that executes queries that don't return results

def run_command(c):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(c)

In [4]:
# A function that returns the list of tables and views in the database. This function will make use of the run_query function

def show_tables():
    table_query = "SELECT name, type FROM sqlite_master WHERE type IN ('table','view');"
    return run_query(table_query)

Now, I will test out the `show_tables` function to be sure that I did the right thing. All things being equal, this should print out all my tables and views in a pandas dataframe.

In [5]:
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


**`QUESTION 1:`** The Chinook record store has just signed a deal with a new record label, and I've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:

|Artist Name | Genre|
|------------|------|
|Regal | Hip-Hop|
|Red Tone | Punk|
|Meteor and the Girls | Pop|
|Slim Jim Bites | Blues|

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

I'll need to write a query to find out which genres sell the most tracks in the USA, and then create a visualization of that data using pandas.

In [6]:
# Query to find the genre which sells the most tracks in the USA
q = '''
WITH usa_tracks_sold AS
   (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )

SELECT
    g.name genre,
    count(uts.invoice_line_id) tracks_sold,
    cast(count(uts.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) from usa_tracks_sold
    ) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN track t on t.track_id = uts.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC;
'''

genre_sales_usa = run_query(q)
run_query(q)

,genre,tracks_sold,percentage_sold
0,Rock,561,0.533777
1,Alternative & Punk,130,0.123692
2,Metal,124,0.117983
3,R&B/Soul,53,0.050428
4,Blues,36,0.034253
5,Alternative,35,0.033302
6,Pop,22,0.020932
7,Latin,22,0.020932
8,Hip Hop/Rap,20,0.019029
9,Jazz,14,0.013321


The result above gives all the different genres we have in our store and the number of tracks each sold in the USA.

From the result above, we can see that the order in which the available genres sold tracks in the US is Punk, Blues, Pop and HipHop.

**Based on this result, I will recommend we purchase albums from the following artists to the store.**
- _Red Tones (Punk)_ 
- _Meteor and the Girls(Blues)_, and 
- _Slim Jim Bites(Pop)_

**`QUESTION 2:`** Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. I have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

I'll like to consider whether any extra columns from the employee table explain any variance I see, or whether the variance might instead be indicative of employee performance.

In [7]:
q2 = """
WITH customer_total AS
    (
    SELECT 
        c.*,
        SUM(i.total) AS total_spend
    FROM invoice i
    INNER JOIN customer c ON i.customer_id = c.customer_id
    GROUP BY c.customer_id
    ORDER BY total_spend DESC
    )
SELECT
    e.employee_id AS Employee_ID,
    e.first_name || ' ' || e.last_name AS Employee_Name,
    e.title AS Title,
    e.hire_date AS Employment_date,
    e.country AS Country,
    SUM(ct.total_spend) AS total_sales
FROM customer_total AS ct
INNER JOIN employee e on e.employee_id = ct.support_rep_id
GROUP by e.employee_id
ORDER BY total_sales DESC
"""
top_sales_agent = run_query(q2)
run_query(q2)

,Employee_ID,Employee_Name,Title,Employment_date,Country,total_sales
0,3,Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,Canada,1731.51
1,4,Margaret Park,Sales Support Agent,2017-05-03 00:00:00,Canada,1584.00
2,5,Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,Canada,1393.92


Using the query, I above selected all the sales that was made and group them by the customer. I further grouped by the sales agent that made the sales since every customer is attached to a sales agent. I calculated the total and ranked it to determine the best performing sales agent.

However, from the results, there just about a 15%-20% difference between the best and least performing sales agents and this can be attributed to their dates of hire as the results also ranked them in orderr of the date they were hired

**`QUESTION 3:`** My next task is to analyze the sales data for customers from each different country. I have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, I have been directed to calculate data, for each country, on the:

- total number of customers
- total value of sales
- average value of sales per customer
- average order value

Because there are a number of countries with only one customer, I am supposed group these customers as "Other" in my analysis. 
The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.

Make one or more recommendations to the marketing team on which countries have potential for growth.

In [8]:
q3= """
WITH cust_sales_by_country AS
    (
    SELECT 
        COUNT(DISTINCT(c.customer_id)) AS no_of_customers,
        SUM(i.total) total_sales,
        SUM(i.total)/COUNT(DISTINCT(c.customer_id)) AS avg_sales_per_customer,
        SUM(i.total)/COUNT(i.total) AS avg_order_value,
        CASE
            WHEN COUNT(DISTINCT(c.customer_id)) <= 1 THEN "Others"
            ELSE c.country
        END AS customer_country
    FROM invoice i
    INNER JOIN customer c ON i.customer_id = c.customer_id
    GROUP BY c.country
    ORDER BY no_of_customers DESC
    )
SELECT
    num_of_customers,
    total_sales,
    avg_sales_per_customer,
    avg_order_value,
    customer_country
FROM
    (
    SELECT 
        SUM(no_of_customers) as num_of_customers,
        total_sales,
        avg_sales_per_customer,
        avg_order_value,
        customer_country,
        CASE 
            WHEN customer_country = 'Others' THEN 1
            ELSE 0
        END AS sort
    FROM cust_sales_by_country csbc
    GROUP BY customer_country
    )
ORDER BY sort, total_sales DESC
"""

run_query(q3)

,num_of_customers,total_sales,avg_sales_per_customer,avg_order_value,customer_country
0,13,1040.49,80.037692,7.942672,USA
1,8,535.59,66.948750,7.047237,Canada
2,5,427.68,85.536000,7.011148,Brazil
3,5,389.07,77.814000,7.781400,France
4,4,334.62,83.655000,8.161463,Germany
5,2,273.24,136.620000,9.108000,Czech Republic
6,3,245.52,81.840000,8.768571,United Kingdom
7,2,185.13,92.565000,6.383793,Portugal
8,2,183.15,91.575000,8.721429,India
9,15,75.24,75.240000,7.524000,Others


In the above query, I answered the question by first grouping the sales made by customer country and calculating the number of customers, total sales, Average sales per customer and average value per order made in each country.

I then go ahead to gather all countries that have only one customer into a group called others and moved it to the bottom of my selection as requested by the stakeholders.

From the above results, I would recommend **Czech Republic** as the country with the most likely potential for growth because despite having just two(2) customers, they have the **_highest average sales per customer_** and also the **_highest average value per order_**. Which means that if we intensify our marketing endeavors in the country and get more customers, there is a high tendency of increased revenue.

**`QUESTION 5:`** The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks.

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase _(unless they do that by choosing each track manually)._ When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

I have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

It is very common when performing an analysis to have **'edge cases'** which prevents you from getting a 100% accurate answer to your question. In this instance, I have two edge cases to consider:

- Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.

- Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue I can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so I can ignore this case also.

In order to answer the question, I'm going to have to identify whether each invoice has all the tracks from an album. I can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. I can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track I pick, since if it's an album purchase, that album will be the same for all tracks.


In [9]:
q4 = """
WITH invoice_first_track AS
    (
    SELECT
        il.invoice_id,
        MIN(il.track_id) AS first_track
    FROM invoice_line il
    GROUP BY il.invoice_id
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
    ift.*,
    CASE
        WHEN
            (
            SELECT t.track_id FROM track t
            WHERE t.album_id = (
                                SELECT t2.album_id FROM track t2
                                WHERE t2.track_id = ift.first_track
                                )
            
            EXCEPT
            
            SELECT il2.track_id FROM invoice_line il2
            WHERE il2.invoice_id = ift.invoice_id
            ) IS NULL
            
        AND
            (
            SELECT il2.track_id FROM invoice_line il2
            WHERE il2.invoice_id = ift.invoice_id
            
            EXCEPT
            
            SELECT t.track_id FROM track t
            WHERE t.album_id = (
                                SELECT t2.album_id FROM track t2
                                WHERE t2.track_id = ift.first_track
                                )
            ) IS NULL
        THEN 'Yes'
        ELSE 'No'
    END AS album_purchase
    FROM invoice_first_track ift
    )
GROUP BY album_purchase;
"""
run_query(q4)

,album_purchase,number_of_invoices,percent
0,No,500,0.814332
1,Yes,114,0.185668


**From the above query, we can see that about 18.6% of the revenue comes from album purchases. I will therefore advise against purchasing only certain tracks from an album as there is a tendency to lose about a fifth of company revenue**

### THE END